In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.cloud import bigquery


In [ ]:
url = "https://www.bankofengland.co.uk/boeapps/database/Bank-Rate.asp"

In [ ]:
# Send a GET request to the URL
response = requests.get(url)

In [ ]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
# Find the table with class "display localtable" 
table = soup.find("table", class_="display localtable")


In [ ]:
# Extract the table headers
headers = [th.text.strip() for th in table.find_all("th")]
headers = [element.replace(" ","_") for element in headers]

In [ ]:
# Extract the table rows
rows = []
for tr in table.find_all("tr"):
    row = [td.text.strip() for td in tr.find_all("td")]
    if row:
        rows.append(row)

In [ ]:
# Create a Pandas DataFrame from the table data
df = pd.DataFrame(rows, columns=headers)
# Convert the date column values to the desired format
df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format="%d %b %y").dt.strftime("%Y-%m-%d")
df

In [ ]:
# Save the edited DataFrame to a new CSV file
output_file = 'edited_boe_rate.csv'
df.to_csv(output_file, index=False)

In [ ]:
# Load the CSV file to BigQuery table
project_id = 'project_id'
dataset_id = 'dataset_id'
table_name = 'table_name'

client = bigquery.Client(project=project_id)
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_name)

job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job_config.autodetect = True  # Infer table schema from CSV file

with open(output_file, 'rb') as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Wait for the job to complete

print(f'Successfully loaded {job.output_rows} rows into {project_id}.{dataset_id}.{table_name}')